In [1]:
from const import CHATGPT_APIKEY, VECTOR_DB_TOKEN, VECTOR_DB_KEYSPACE, VECTOR_DB_SCB, VECTOR_DB_SCB_PATH
import os
import openai
from langchain.vectorstores.cassandra import  Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings

from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

from datasets import load_dataset

print(VECTOR_DB_TOKEN)

Hello
{'clientId': 'hcmuhyiMcXuPzUDvWgIrukuJ', 'secret': '2BRqrp92ezh5rGzmX1-+3ovEbDt1S10fKNL8GJ.HC7avsx.dAtoe_vbHUwmsMtw,S1OJTDYGtS0.j-mhys8x+qZt-InNrmZCFhP7RHFN51zqi-jlJRut2j9tptnd72OC', 'token': 'AstraCS:hcmuhyiMcXuPzUDvWgIrukuJ:307590ad8309ca349b99632d92380169a7fb73c7624dc5d207637198941bf0f6'}


In [ ]:



print("KarlBot")

openai.api_key = CHATGPT_APIKEY

response = openai.Embedding.create(
    input="The food was delicious",
    model="text-embedding-ada-002"
)
embeddings = response['data'][0]['embedding']


In [ ]:
print(response)

In [3]:

auth_provider = PlainTextAuthProvider(VECTOR_DB_TOKEN["clientId"], VECTOR_DB_TOKEN["secret"])

cloudConfig = {
  "secure_connect_bundle": VECTOR_DB_SCB_PATH
}
cluster = Cluster(cloud=cloudConfig, auth_provider=auth_provider)
astraSession = cluster.connect()


llm = OpenAI(openai_api_key=CHATGPT_APIKEY)
myEmbedding = OpenAIEmbeddings(openai_api_key=CHATGPT_APIKEY)

cassandraVStore = Cassandra(
  embedding=myEmbedding,
  session=astraSession,
  keyspace=VECTOR_DB_KEYSPACE,
  table_name="qa_mini_demo"
)

print("Loading data from huggingface")
myDataSet = load_dataset("Biddls/Onion_News", split="train")
headlines = myDataSet["text"][:50]

print("Generating embeddings and storing in AstraDB")
cassandraVStore.add_texts(headlines)

print("Inserted %i headlines. \n" % len(headlines))


Loading data from huggingface


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating embeddings and storing in AstraDB
Inserted 50 headlines. 



In [6]:
vectorIndx = VectorStoreIndexWrapper(vectorstore=cassandraVStore)

while True:
  queryTxt = input("Enter your question (or type quit to exit): ")

  if queryTxt.lower() == "quit":
    break

  print("QUESTION: \"%s\"" % queryTxt)
  ans = vectorIndx.query(queryTxt, llm=llm).strip()
  print(f'ANSWER: {ans}\n')
  print(f'DOCUMENTS BY RELEVANCE:')
  for doc, score in cassandraVStore.similarity_search_with_score(queryTxt, k=4):
    print(f'  {score}0.4f {doc.page_content[:60]}   ')

QUESTION: "What is love"
ANSWER: I don't know.

DOCUMENTS BY RELEVANCE:
  0.86481447911669410.4f ‘I Love That It Has Pockets,’ Says Woman Showing Off New Boy   
  0.86265793547133620.4f Pope Francis Declares Nothing Wrong With Guy Giving Buddy Tu   
  0.86243768669523520.4f Astronaut Buzz Aldrin Marries Longtime Hallucination Of Sexy   
  0.8621599305607750.4f Man Buys Slice Of Honey-Roasted Ham For Attractive Woman At    
QUESTION: "Where will I be working in 10 years"
ANSWER: I don't know.

DOCUMENTS BY RELEVANCE:
  0.87630018299545620.4f Look What Happens When You Leave A McDonald’s Hamburger Out    
  0.8750482441020020.4f Doomsday Clock Moves 10 Seconds Closer To Midnight #~# The D   
  0.8733421689134960.4f ChatGPT Forced To Take Bar Exam Even Though Dream Was To Be    
  0.87313859653617590.4f Google Employees React To Company Layoffs #~# After Google r   
